### Main notebook

In [1]:
from scraper import DataLoader

### Scrape google map's reviews into dataframe

In [2]:
# insert goole maps business url
url = 'https://maps.app.goo.gl/geQwCeu2riPkrSqt8'


In [3]:
# Loader init
loader = DataLoader(url)

In [4]:
# navigate to the reviews page
loader.navigate()

In [10]:
loader.create_reviews_df()
loader.df.stars.mean()

4.3

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()  # Or whichever browser/driver you're using
# define url
url = 'https://maps.app.goo.gl/geQwCeu2riPkrSqt8'
driver.get(url)

In [3]:
from bs4 import BeautifulSoup

# Finding the address of the location
response = BeautifulSoup(driver.page_source, 'html.parser')
address = response.find('div',class_= 'rogA2c').text

In [4]:
xpath = "//button[contains(., 'ביקורות') or contains(., 'REVIEWS')]"
wait = WebDriverWait(driver, 10)
route_button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
route_button.click()

In [1]:
import os
import pandas as pd

from glob import glob


In [2]:
home_dir = 'c:\\Users\\DanaTal\\projects\\nlp\\Bridezilla_NLP'
os.path.exists(home_dir)
data_dir = os.path.join(home_dir, f'data')
data_path = glob(data_dir + '\\*.csv')[0]
data_path


'c:\\Users\\DanaTal\\projects\\nlp\\Bridezilla_NLP\\data\\reviews_Old-Phoenix Rooms & Restaurant.csv'

Load Reviews

In [19]:
df = pd.read_csv(data_path)
print(f' number of reviews is: {len(df)}')
df.head()

 number of reviews is: 189


,text,stars,months_ago
0,We love The Old Phoenix - this is the fourth t...,5,5
1,What a disappointment. We have wanted to stay ...,1,4
2,We walked over to Fenix from Loutro and stayed...,5,6
3,The Old Phoenix was the low point of our two w...,1,5
4,I had a fantastic time at Old-Phoenix!\nThe be...,5,7


In [21]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

In [24]:
# Load Tokenizer & model
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model =  AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

c:\Users\DanaTal\anaconda3\envs\dana\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
c:\Users\DanaTal\anaconda3\envs\dana\Lib\site-packages\transformers\models\auto\modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelFo

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Infer one review

In [36]:
# t5 can handle once 512 tokens
sequence = (df.text[0])
sequence

'We love The Old Phoenix - this is the fourth time we’ve visited in the last ten years or so. It’s secluded and quiet, and the swimming in the bay is sublime. If you’re expecting five-star luxury, go somewhere else; this is a traditional …'

In [29]:
# tokenize one review
inputs = tokenizer.encode("summarize: " + sequence, return_tensors='pt', max_length=512, truncation=True)

In [31]:
inputs

tensor([[21603,    10,   101,   333,    37,  3525, 12308,     3,    18,    48,
            19,     8,  4509,    97,    62,    22,   162,  5251,    16,     8,
           336,     3,   324,   203,    42,    78,     5,    94,    22,     7,
             3,     7, 28426,    11,  4354,     6,    11,     8,  5989,    16,
             8, 10210,    19, 25781,     5,   156,    25,    22,    60, 11873,
           874,    18,  3624,  3873,     6,   281,  5775,  1307,   117,    48,
            19,     3,     9,  1435,     3,   233,     1]])

In [43]:
# this will generate a numerical output
output = model.generate(inputs, max_length=40, min_length=1, length_penalty=5, num_beams=2)
output

tensor([[    0,     8,  1595,    19,     8,  4509,    62,    31,   162,  5251,
            16,     8,   336,     3,   324,   203,    42,    78,     3,     5,
            34,    31,     7,     3,     7, 28426,    11,  4354,     6,    11,
             8,  5989,    16,     8, 10210,    19, 25781,     3,     5,     3]])

In [44]:
print(tokenizer.decode(output[0]))

<pad> the hotel is the fourth we've visited in the last ten years or so. it's secluded and quiet, and the swimming in the bay is sublime. 


### Take all the reviews

In [53]:
sequence = tuple(df.text.tolist())
print(sequence)

('We love The Old Phoenix - this is the fourth time we’ve visited in the last ten years or so. It’s secluded and quiet, and the swimming in the bay is sublime. If you’re expecting five-star luxury, go somewhere else; this is a traditional …', 'What a disappointment. We have wanted to stay here for years but it was not a nice experience. Fine for a day at the beach and lunch at the taverna but don’t expect friendly service, the waiters couldn’t care less. And do not stay here! …', 'We walked over to Fenix from Loutro and stayed the day on the beach by the Old Phoenix. Wonderful small stony beach, clean clear water and very quiet. Had lunch in the restaurant that was very good, fresh local ingredients and lovely …', 'The Old Phoenix was the low point of our two week vacation. Our room looked nothing like the photos. No art on the walls, fluorescent blue lights (and fluorescent reading lights, well, only one worked, that were brighter than the …', 'I had a fantastic time at Old-Phoenix!\n

In [54]:
# tokenize one review
inputs = tokenizer.encode("summarize: " + sequence, return_tensors='pt', max_length=512, truncation=True)

TypeError: can only concatenate str (not "tuple") to str